In [67]:
import os
import pandas as pd
import numpy as np
import re
from datetime import timedelta # Although fiscal date processing is currently skipped

# --- Configuration ---
# Define base data path and years to process
DATA_PATH = r"C:\Users\yadla\OneDrive\Desktop\Hospital_Analysis_Project\Data"
YEARS = range(2018, 2023) # 2018–2022 inclusive

# Paths for external data files (Inflation and Processed Medicare Data)
INFLATION_DATA_FILE = r"C:\Users\yadla\OneDrive\Desktop\Hospital_Analysis_Project\Inflation Data\file.csv" # Path to your simple yearly inflation file
PROCESSED_MEDICARE_FILE = r"C:/Users/yadla/OneDrive/Desktop/Hospital_Analysis_Project/Medicare Data/Medicare_Enrollment_Processed.csv" # Path to your processed Medicare file

# Define the final output path for the integrated, imputed subset file
FINAL_OUTPUT_FILE_PATH = os.path.join(DATA_PATH, "Hospital_Financial_Analysis_Subset_Integrated_Imputed.csv")

# Columns to drop in the initial cleaning phase
INITIAL_COLS_TO_DROP = [
    "Hospital Total Days Title V For Adults & Peds",
    "Hospital Total Days Title XIX For Adults & Peds",
    "Hospital Total Days Title XVIII For Adults & Peds",
]

# Rename map for columns
RENAME_MAP = {
    "Less: Allowances for Uncollectible Notes and Accounts Receivable": (
        "Allowances for Uncollectible Notes and Accounts Receivable"
    ),
    "Less Contractual Allowance and Discounts on Patients' Accounts": (
        "Contractual Allowance and Discounts on Patients' Accounts"
    ),
    # Added renaming for the total hospital days/visits column
    "Hospital Total Days (V + XVIII + XIX + Unknown) For Adults & Peds": "Hospital Total Days Or Visits",
}

# Define the exact list of hospital financial columns to select for the final subset
# 'Rural Versus Urban' is INCLUDED in this list so it's in the output file.
COLUMNS_FOR_FINAL_SUBSET = [
    "Year",
    "facility_id",
    "Total Revenue",
    "Hospital Name",
    "State Code",
    "County",
    "Rural Versus Urban", # Included for visualization/EDA
    "Total_Staffing_FTE", # Combined staffing column
    "Number of Beds",
    "Cost of Charity Care",
    "Net Revenue from Medicaid",
    "Total Bad Debt Expense",
    "Other Assets",
    "Cost of Uncompensated Care",
    "Total Unreimbursed and Uncompensated Care",
    "Cost To Charge Ratio",
    "Prepaid Expenses",
    "Other Current Liabilities",
    "Total Other Assets",
    "Total Long Term Liabilities",
    "Buildings",
    "Salaries, Wages, and Fees Payable",
    "Inventory",
    "Major Movable Equipment",
    "Outpatient Revenue",
    "Cash on Hand and in Banks",
    "Outpatient Total Charges",
    "Accounts Payable",
    "Accounts Receivable",
    "Total Fixed Assets",
    "Contractual Allowance and Discounts on Patients' Accounts",
    "Total Current Liabilities",
    "Total Liabilities",
    "Total Current Assets",
    "General Fund Balance",
    "Total Fund Balances",
    "Total Liabilities and Fund Balances",
    "Total Assets",
    "Total Other Income",
    "Inpatient Revenue",
    "Total Patient Revenue",
    "Net Patient Revenue",
    "Depreciation Cost",
    "Total Income",
    "Net Income",
    "Less Total Operating Expense",
    "Net Income from Service to Patients",
    "Inpatient Total Charges",
    "Total Salaries From Worksheet A",
    "Overhead Non-Salary Costs",
    "Total Costs",
    "Combined Outpatient + Inpatient Total Charges",
    # Now requesting the RENAMED column name for the final subset selection
    "Hospital Total Days Or Visits",
    # Note: Columns from external data ('Annual', 'Total_Yearly_Medicare_Enrollment')
    # will be added to the DataFrame during processing and are not listed here
    # as they are not part of the *initial* selection list.
]

# Define the base categorization dictionary for the initial financial columns
# 'Rural Versus Urban' is INCLUDED in this map.
column_category_map = {
    "Year": {'category': 'Time', 'subcategory': None},
    "facility_id": {'category': 'Identifier', 'subcategory': None},
    "Hospital Name": {'category': 'Identifier', 'subcategory': None},
    "State Code": {'category': 'Geography', 'subcategory': 'State'},
    "County": {'category': 'Geography', 'subcategory': 'County'},
    "Rural Versus Urban": {'category': 'Geography', 'subcategory': 'Rural/Urban'}, # Included for categorization
    "Total_Staffing_FTE": {'category': 'Operations', 'subcategory': 'Staffing'},
    "Number of Beds": {'category': 'Operations', 'subcategory': 'Capacity'},
    # Updated category map entry for the renamed column
    "Hospital Total Days Or Visits":{'category': 'Operations', 'subcategory': 'Capacity'},

    "Inpatient Revenue": {'category': 'Revenue', 'subcategory': 'Gross Patient Revenue'},
    "Outpatient Revenue": {'category': 'Revenue', 'subcategory': 'Gross Patient Revenue'},
    "Total Patient Revenue": {'category': 'Revenue', 'subcategory': 'Gross Patient Revenue'},
    "Net Patient Revenue": {'category': 'Revenue', 'subcategory': 'Net Patient Revenue'},
    "Total Other Income": {'category': 'Revenue', 'subcategory': 'Other Revenue'},
    "Total Income": {'category': 'Revenue', 'subcategory': 'Total Income'},
    "Total Revenue": {'category': 'Revenue', 'subcategory': 'Total Revenue'},

    "Less Total Operating Expense": {'category': 'Expenses & Costs', 'subcategory': 'Operating Expenses (Total)'},
    "Contractual Allowance and Discounts on Patients' Accounts": {'category': 'Expenses & Costs', 'subcategory': 'Costs'},
    "Salaries, Wages, and Fees Payable": {'category': 'Expenses & Costs', 'subcategory': 'Costs'}, # Also a Liability
    "Total Salaries From Worksheet A": {'category': 'Expenses & Costs', 'subcategory': 'Costs'},
    "Overhead Non-Salary Costs": {'category': 'Expenses & Costs', 'subcategory': 'Costs'},
    "Depreciation Cost": {'category': 'Expenses & Costs', 'subcategory': 'Costs'},
    "Total Costs": {'category': 'Expenses & Costs', 'subcategory': 'Costs'},
    "Cost of Charity Care": {'category': 'Expenses & Costs', 'subcategory': 'Uncompensated Care Costs'},
    "Total Bad Debt Expense": {'category': 'Expenses & Costs', 'subcategory': 'Uncompensated Care Costs'},
    "Cost of Uncompensated Care": {'category': 'Expenses & Costs', 'subcategory': 'Uncompensated Care Costs'},
    "Total Unreimbursed and Uncompensated Care": {'category': 'Expenses & Costs', 'subcategory': 'Uncompensated Care Costs'},

    "Net Income": {'category': 'Profitability', 'subcategory': None},
    "Net Income from Service to Patients": {'category': 'Profitability', 'subcategory': 'Income from Patient Services'},

    "Inpatient Total Charges": {'category': 'Charges', 'subcategory': None},
    "Outpatient Total Charges": {'category': 'Charges', 'subcategory': None},
    "Combined Outpatient + Inpatient Total Charges": {'category': 'Charges', 'subcategory': None},

    "Cash on Hand and in Banks": {'category': 'Balance Sheet - Assets', 'subcategory': 'Current Assets'},
    "Inventory": {'category': 'Balance Sheet - Assets', 'subcategory': 'Current Assets'},
    "Accounts Receivable": {'category': 'Balance Sheet - Assets', 'subcategory': 'Current Assets'},
    "Prepaid Expenses": {'category': 'Balance Sheet - Assets', 'subcategory': 'Current Assets'},
    "Total Current Assets": {'category': 'Balance Sheet - Assets', 'subcategory': 'Current Assets'},
    "Buildings": {'category': 'Balance Sheet - Assets', 'subcategory': 'Fixed Assets'},
    "Major Movable Equipment": {'category': 'Balance Sheet - Assets', 'subcategory': 'Fixed Assets'},
    "Total Fixed Assets": {'category': 'Balance Sheet - Assets', 'subcategory': 'Fixed Assets'},
    "Other Assets": {'category': 'Balance Sheet - Assets', 'subcategory': 'Other Assets'},
    "Total Other Assets": {'category': 'Balance Sheet - Assets', 'subcategory': 'Other Assets'},
    "Total Assets": {'category': 'Balance Sheet - Assets', 'subcategory': 'Total Assets'},

    "Accounts Payable": {'category': 'Balance Sheet - Liabilities', 'subcategory': 'Current Liabilities'},
    "Salaries, Wages, and Fees Payable": {'category': 'Balance Sheet - Liabilities', 'subcategory': 'Current Liabilities'}, # Listed again as it's a liability
    "Other Current Liabilities": {'category': 'Balance Sheet - Liabilities', 'subcategory': 'Current Liabilities'},
    "Total Current Liabilities": {'category': 'Balance Sheet - Liabilities', 'subcategory': 'Current Liabilities'},
    "Total Long Term Liabilities": {'category': 'Balance Sheet - Liabilities', 'subcategory': 'Long-Term Liabilities'},
    "Total Liabilities": {'category': 'Balance Sheet - Liabilities', 'subcategory': 'Total Liabilities'},

    "General Fund Balance": {'category': 'Balance Sheet - Fund Balance/Equity', 'subcategory': None},
    "Total Fund Balances": {'category': 'Balance Sheet - Fund Balance/Equity', 'subcategory': None},
    "Total Liabilities and Fund Balances": {'category': 'Balance Sheet - Fund Balance/Equity', 'subcategory': None},

    "Net Revenue from Medicaid": {'category': 'Payer & Reimbursement Specific', 'subcategory': 'Medicaid Reimbursement'},

    "Cost To Charge Ratio": {'category': 'Key Ratios', 'subcategory': None},
}


# --- Helper Functions ---

def load_and_combine_data(data_path, years):
    """
    Loads and combines CSV files from specified years into a single DataFrame.
    Adds a 'Year' column to each DataFrame.

    Args:
        data_path (str): The path to the directory containing the yearly CSV files.
        years (range): A range of years to load data for.

    Returns:
        pd.DataFrame: A concatenated DataFrame containing data from all specified years,
                      or an empty DataFrame if no files were loaded.
    """
    dfs = []
    print("--- Step 1: Loading and Combining Data ---")
    for yr in years:
        file_path = os.path.join(data_path, f"Hospital_{yr}.csv")
        try:
            # Use low_memory=False to avoid DtypeWarning with mixed types
            df_year = pd.read_csv(file_path, low_memory=False)
            df_year["Year"] = yr
            dfs.append(df_year)
            print(f"Successfully loaded data for year {yr}.")
        except FileNotFoundError:
            print(f"Error: File not found at {file_path}. Skipping.")
        except Exception as e:
            print(f"An error occurred while processing {file_path}: {e}. Skipping.")

    if dfs:
        master_df = pd.concat(dfs, ignore_index=True)
        print(f"Combined data from {len(dfs)} years into a master DataFrame (initial load).")
        return master_df
    else:
        print("No dataframes were loaded.")
        return pd.DataFrame() # Return empty DataFrame if no data loaded

def clean_column_names(df):
    """
    Strips leading and trailing whitespace from all column names in the DataFrame.

    Args:
        df (pd.DataFrame): The input DataFrame.

    Returns:
        pd.DataFrame: The DataFrame with cleaned column names.
    """
    if not df.empty:
        print("--- Step 2: Cleaning Column Names ---")
        df.columns = df.columns.str.strip()
        print("Stripped whitespace from column names.")
    return df

def clean_data_values(df):
    """
    Removes specific characters (',', '.', ' ') only from the start or end
    of string values in object columns. Preserves characters within the string.

    Args:
        df (pd.DataFrame): The input DataFrame.

    Returns:
        pd.DataFrame: The DataFrame with cleaned string values.
    """
    if not df.empty:
        print("--- Step 3: Cleaning Data Values ---")
        chars_to_strip = ",'. " # Characters to remove from start/end
        for col in df.columns:
            # Apply only to columns that are of object (string) dtype
            if df[col].dtype == 'object':
                # Convert to string first to handle potential non-string data gracefully
                # Apply strip method to remove leading/trailing characters
                df[col] = df[col].astype(str).str.strip(chars_to_strip)
        print(f"Removed leading/trailing characters ('{chars_to_strip}') from string columns.")
    return df

def process_fiscal_dates(df):
    """
    Converts 'Fiscal Year Begin Date' and 'Fiscal Year End Date' to datetime objects
     and extracts the year from the begin date into a new 'Fiscal Year' column.
    Assumes date format is DD/MM/YYYY.
    Note: This function is included for completeness but is currently skipped
    in the main execution flow as per previous instructions.

    Args:
        df (pd.DataFrame): The input DataFrame.

    Returns:
        pd.DataFrame: The DataFrame with processed fiscal date columns (if present).
    """
    begin_col = 'Fiscal Year Begin Date'
    end_col = 'Fiscal Year End Date'

    if not df.empty and begin_col in df.columns and end_col in df.columns:
        print(f"--- Attempting to Process Fiscal Dates ('{begin_col}', '{end_col}') ---")
        try:
            # Convert to datetime objects, specifying the format
            df[begin_col] = pd.to_datetime(df[begin_col], format='%d/%m/%Y', errors='coerce')
            df[end_col] = pd.to_datetime(df[end_col], format='%d/%m/%Y', errors='coerce')
            print("Converted fiscal date columns to datetime objects.")

            # Extract the year from the begin date
            df['Fiscal Year'] = df[begin_col].dt.year
            print("Created 'Fiscal Year' column from begin date.")

            # Optional: Handle cases where conversion failed (NaNs introduced by errors='coerce')
            if df[begin_col].isnull().any() or df[end_col].isnull().any():
                 print("Warning: Some fiscal date entries could not be parsed and resulted in NaNs.")

        except Exception as e:
            print(f"Error processing fiscal date columns: {e}")
            print("Skipping fiscal date processing.")

    elif not df.empty:
        missing_cols = [col for col in [begin_col, end_col] if col not in df.columns]
        print(f"Skipping fiscal date processing. Missing columns: {missing_cols}")

    return df


def drop_specified_columns(df, columns_to_drop):
    """
    Drops a list of specified columns from the DataFrame, ignoring columns not found.

    Args:
        df (pd.DataFrame): The input DataFrame.
        columns_to_drop (list): A list of column names to drop.

    Returns:
        pd.DataFrame: The DataFrame with specified columns dropped.
    """
    if not df.empty and columns_to_drop:
        print("--- Step 4: Dropping Specified Columns ---")
        initial_cols = set(df.columns)
        df = df.drop(columns=columns_to_drop, errors="ignore")
        dropped_cols_actual = initial_cols - set(df.columns)
        if dropped_cols_actual:
            print(f"Dropped columns: {list(dropped_cols_actual)}")
        else:
            present_cols_to_drop = [col for col in columns_to_drop if col in initial_cols]
            if present_cols_to_drop:
                 print(f"Attempted to drop columns {present_cols_to_drop}, but they were not found after previous steps.")
            else:
                 print(f"Attempted to drop columns {columns_to_drop}, but none were found in the DataFrame.")
    elif not df.empty:
        print("--- Step 4: No columns specified to drop ---")
    return df

def drop_rows_missing_cbsa(df):
    """
    Drops rows where 'Medicare CBSA Number' is missing or empty.

    Args:
        df (pd.DataFrame): The input DataFrame.

    Returns:
        pd.DataFrame: The DataFrame with rows containing missing CBSA dropped.
    """
    if not df.empty and "Medicare CBSA Number" in df.columns:
        print("--- Step 5: Dropping Rows with Missing CBSA ---")
        initial_rows = len(df)
        # Ensure the column is treated as string for stripping and check for non-empty strings
        df = df[
            df["Medicare CBSA Number"].notna()
            & (df["Medicare CBSA Number"].astype(str).str.strip() != "")
        ]
        rows_dropped = initial_rows - len(df)
        print(f"Dropped {rows_dropped} rows with missing or empty 'Medicare CBSA Number'. Remaining rows: {len(df)}")
    elif not df.empty:
         print("--- Step 5: Skipping dropping rows with missing CBSA as 'Medicare CBSA Number' column is missing or DataFrame is empty ---")
    return df

def create_derived_financial_columns(df):
    """
    Combines staffing FTE columns into a single 'Total_Staffing_FTE' column
    and creates a 'Total Revenue' column by summing 'Total Patient Revenue'
    and 'Total Other Income'.

    Args:
        df (pd.DataFrame): The input DataFrame.

    Returns:
        pd.DataFrame: The DataFrame with the new derived columns.
    """
    if df.empty:
        print("--- Step 6: Skipping Derived Column Creation - DataFrame is empty ---")
        return df

    print("--- Step 6: Creating Derived Financial Columns (Staffing FTE, Total Revenue) ---")

    # Combine staffing FTE
    fte_cols = ["FTE - Employees on Payroll", "Number of Interns and Residents (FTE)"]
    combined_fte_col_name = "Total_Staffing_FTE"
    present_fte_cols = [col for col in fte_cols if col in df.columns]

    if present_fte_cols:
        print(f"  Combining staffing FTE columns: {present_fte_cols} into '{combined_fte_col_name}'")
        df_fte_subset = df[present_fte_cols].copy()
        # Convert to numeric, coercing errors to NaN, then fill NaN with 0 for summation
        for col in df_fte_subset.columns:
            df_fte_subset[col] = pd.to_numeric(df_fte_subset[col], errors='coerce').fillna(0)
        df[combined_fte_col_name] = df_fte_subset.sum(axis=1)
        print(f"  Created '{combined_fte_col_name}' column.")
        # Drop original FTE columns if they exist
        df = df.drop(columns=present_fte_cols, errors='ignore')
        print(f"  Dropped original staffing FTE columns: {present_fte_cols}")
    else:
        print(f"  None of the required staffing FTE columns ({fte_cols}) were found. Skipping combination.")
        # Create the combined column with NaNs if components are missing
        df[combined_fte_col_name] = np.nan


    # Create Total Revenue column
    revenue_cols = ["Total Patient Revenue", "Total Other Income"]
    total_revenue_col_name = "Total Revenue"
    if all(col in df.columns for col in revenue_cols):
        print(f"  Creating '{total_revenue_col_name}' column.")
        # Convert to numeric, coercing errors to NaN, before summation
        df["Total Patient Revenue"] = pd.to_numeric(df["Total Patient Revenue"], errors='coerce')
        df["Total Other Income"] = pd.to_numeric(df["Total Other Income"], errors='coerce')
        # Summing will result in NaN if either component is NaN
        df[total_revenue_col_name] = df["Total Patient Revenue"] + df["Total Other Income"]
        print(f"  Created '{total_revenue_col_name}' column.")
    else:
         print(f"  Skipping Total Revenue creation. One or more required columns ({revenue_cols}) are missing or not numeric.")

    return df


def create_facility_id(df):
    """
    Creates a unique identifier for each facility using Hospital Name, City,
    State Code, Provider Type, and Provider CCN.

    Args:
        df (pd.DataFrame): The input DataFrame.

    Returns:
        pd.DataFrame: The DataFrame with the 'facility_id' column added.
    """
    if not df.empty:
        print("--- Step 7: Creating Facility ID ---")
        id_cols = ["Hospital Name", "City", "State Code", "Provider Type", "Provider CCN"]
        if all(col in df.columns for col in id_cols):
            # Ensure columns are string type before applying string methods and strip whitespace
            for col in id_cols:
                df[col] = df[col].astype(str).str.strip()

            # Create the unique facility ID by concatenating lowercased and stripped components
            df["facility_id"] = (
                df["Hospital Name"].str.lower()
                + "_"
                + df["City"].str.lower()
                + "_"
                + df["State Code"].str.lower()
                + "_"
                + df["Provider Type"].str.lower()
                + "_"
                + df["Provider CCN"].str.lower() # Include Provider CCN for uniqueness
            )
            print("Created 'facility_id' column.")
        else:
            missing_cols = [col for col in id_cols if col not in df.columns]
            print(f"Skipping facility_id creation. One or more required columns ({missing_cols}) are missing.")
            # If essential ID columns are missing, facility_id cannot be created.
            # This might impact subsequent steps that rely on facility_id.
    else:
        print("--- Step 7: Skipping Facility ID Creation - DataFrame is empty ---")
    return df

def filter_facilities_all_years(df, required_years):
    """
    Keeps only facilities that reported data in all specified required years.

    Args:
        df (pd.DataFrame): The input DataFrame with 'facility_id' and 'Year' columns.
        required_years (range): A range of years that facilities must have data for.

    Returns:
        pd.DataFrame: A filtered DataFrame containing only facilities present in all
                      required years, or an empty DataFrame if filtering cannot be applied
                      or results in no facilities.
    """
    if not df.empty and "facility_id" in df.columns and "Year" in df.columns:
        print(f"--- Step 8: Filtering for Facilities in All {len(required_years)} Years ---")
        # Count the number of unique years for each facility
        year_counts = df.groupby("facility_id")["Year"].nunique()
        # Identify facility IDs that have data for the exact number of required years
        valid_ids = year_counts[year_counts == len(required_years)].index
        # Filter the DataFrame to keep only rows corresponding to valid facility IDs
        df_filtered = df[df["facility_id"].isin(valid_ids)].copy() # Use .copy() to avoid SettingWithCopyWarning
        print(f"Filtered for facilities reporting in all {len(required_years)} file years. Remaining unique facilities: {df_filtered['facility_id'].nunique()}")
        return df_filtered
    elif not df.empty:
         print("--- Step 8: Skipping filtering for facilities in all years due to missing 'facility_id' or 'Year' column or DataFrame is empty ---")
    return pd.DataFrame() # Return empty DataFrame if filtering cannot be applied or input is empty

def rename_columns(df, rename_map):
    """
    Renames columns in the DataFrame based on a provided mapping.
    Only attempts to rename columns that exist in the DataFrame.

    Args:
        df (pd.DataFrame): The input DataFrame.
        rename_map (dict): A dictionary where keys are old column names and values
                           are new column names.

    Returns:
        pd.DataFrame: The DataFrame with columns renamed according to the map.
    """
    if not df.empty and rename_map:
        print("--- Step 9: Renaming Columns ---")
        # Filter rename_map to only include columns actually in the DataFrame
        valid_rename_map = {old_name: new_name for old_name, new_name in rename_map.items() if old_name in df.columns}
        if valid_rename_map:
            df = df.rename(columns=valid_rename_map)
            print(f"Renamed specified columns: {list(valid_rename_map.keys())}")
        else:
            print("No columns found in DataFrame matching the rename map.")
    elif not df.empty:
        print("--- Step 9: Rename map is empty. No columns renamed ---")
    return df


def load_and_process_inflation_data(file_path):
    """
    Loads and processes simple yearly inflation data from a CSV file.
    Expects columns 'Year' and 'Annual'. Converts 'Year' to integer and
    'Annual' to numeric. Drops rows with missing essential data.

    Args:
        file_path (str): The path to the inflation data CSV file.

    Returns:
        pd.DataFrame: A processed DataFrame with 'Year' and 'Annual' columns,
                      or an empty DataFrame if loading or processing fails.
    """
    print(f"--- Step 10: Loading and Processing Inflation Data from: {file_path} ---")
    try:
        df_inflation = pd.read_csv(file_path, low_memory=False)
        print("Successfully loaded inflation data.")
        # Ensure 'Year' and 'Annual' columns exist and are of correct types
        if 'Year' in df_inflation.columns and 'Annual' in df_inflation.columns:
            # Use Int64 for nullable integer year
            df_inflation['Year'] = pd.to_numeric(df_inflation['Year'], errors='coerce').astype('Int64')
            df_inflation['Annual'] = pd.to_numeric(df_inflation['Annual'], errors='coerce')
            # Drop rows where Year or Annual are missing after conversion
            df_inflation = df_inflation.dropna(subset=['Year', 'Annual']).copy() # Use .copy()
            # Select only the relevant columns
            df_inflation_processed = df_inflation[['Year', 'Annual']].copy() # Use .copy()
            print("Inflation data processed (selected 'Year' and 'Annual', converted types).")
            return df_inflation_processed
        else:
            print("Error: Required columns ('Year' or 'Annual') not found in Inflation Data. Returning empty DataFrame.")
            return pd.DataFrame()
    except FileNotFoundError:
        print(f"Error: Inflation Data file not found at {file_path}. Returning empty DataFrame.")
        return pd.DataFrame()
    except Exception as e:
        print(f"An error occurred while loading/processing Inflation Data from {file_path}: {e}. Returning empty DataFrame.")
        return pd.DataFrame()

def load_and_process_medicare_data(file_path):
    """
    Loads and processes the already processed Medicare enrollment data from a CSV file.
    Expects columns 'YEAR', 'BENE_STATE_ABRVTN', 'TOT_BENES'. Aggregates data
    to get yearly total beneficiaries per state. Renames columns for merging.

    Args:
        file_path (str): The path to the processed Medicare enrollment CSV file.

    Returns:
        pd.DataFrame: An aggregated and processed DataFrame with yearly total
                      Medicare enrollment per state, or an empty DataFrame
                      if loading or processing fails.
    """
    print(f"--- Step 11: Loading and Processing Medicare Enrollment Data from: {file_path} ---")
    try:
        df_medicare = pd.read_csv(file_path, low_memory=False)
        print("Successfully loaded processed Medicare enrollment data.")

        # Ensure required columns exist and are of correct types
        required_cols = ['YEAR', 'BENE_STATE_ABRVTN', 'TOT_BENES']
        if all(col in df_medicare.columns for col in required_cols):
            # Convert types, coercing errors to NaN
            df_medicare['YEAR'] = pd.to_numeric(df_medicare['YEAR'], errors='coerce').astype('Int64') # Use Int64 for nullable integer
            df_medicare['TOT_BENES'] = pd.to_numeric(df_medicare['TOT_BENES'], errors='coerce')
            df_medicare['BENE_STATE_ABRVTN'] = df_medicare['BENE_STATE_ABRVTN'].astype(str).str.strip()

            # Drop rows with missing essential data for aggregation
            df_medicare_agg = df_medicare.dropna(subset=required_cols).copy() # Use .copy()

            if not df_medicare_agg.empty:
                # Aggregate Medicare data to yearly total beneficiaries per state
                print("Aggregating Medicare data to yearly total beneficiaries per state...")
                df_medicare_yearly = df_medicare_agg.groupby(['YEAR', 'BENE_STATE_ABRVTN'])['TOT_BENES'].sum().reset_index()

                # Rename columns to match hospital data for merging
                df_medicare_yearly = df_medicare_yearly.rename(columns={
                    'YEAR': 'Year',
                    'BENE_STATE_ABRVTN': 'State Code', # Assuming State Code in hospital data is abbreviation
                    'TOT_BENES': 'Total_Yearly_Medicare_Enrollment' # New column name
                })
                print("Medicare data aggregated and columns renamed.")
                return df_medicare_yearly
            else:
                print("Processed Medicare data is empty after dropping NaNs for aggregation. Returning empty DataFrame.")
                return pd.DataFrame()
        else:
            missing = [col for col in required_cols if col not in df_medicare.columns]
            print(f"Error: Required columns for Medicare processing ({missing}) not found. Returning empty DataFrame.")
            return pd.DataFrame()

    except FileNotFoundError:
        print(f"Error: Processed Medicare Enrollment file not found at {file_path}. Returning empty DataFrame.")
        return pd.DataFrame()
    except Exception as e:
        print(f"An error occurred while loading/processing Medicare Enrollment Data from {file_path}: {e}. Returning empty DataFrame.")
        return pd.DataFrame()


def impute_missing_financial_data(df):
    """
    Imputes missing values in numeric columns based on the specified strategy:
    1. Median of remaining years for the facility (if 1 or 2 missing years out of 5).
    2. Median of the county for that year (if > 2 missing years or facility median not available).
    3. Median of the state for that year (if county median not available).

    Args:
        df (pd.DataFrame): The input DataFrame with numeric columns to impute.

    Returns:
        pd.DataFrame: The DataFrame with missing numeric values imputed.
    """
    if df.empty:
        print("--- Step 15: Skipping Imputation - DataFrame is empty ---")
        return df

    print("\n--- Step 15: Starting Missing Value Imputation ---")

    # Identify numeric columns to impute (exclude identifier/grouping columns)
    # Include columns from external data if they are numeric and need imputation
    cols_to_impute = df.select_dtypes(include=np.number).columns.tolist()
    # Ensure identifier/grouping columns are excluded if they are numeric (Year, facility_id, etc.)
    # Add 'State Code', 'County' to exclude_list if they are numeric in your df
    cols_to_exclude = ['Year', 'facility_id']

    cols_to_impute = [col for col in cols_to_impute if col not in cols_to_exclude]

    if not cols_to_impute:
        print("No numeric columns found to impute. Skipping imputation.")
        return df

    initial_missing_count = df[cols_to_impute].isnull().sum().sum()
    print(f"Initial total missing values in numeric columns to impute: {initial_missing_count}")


    for col in cols_to_impute:
        # --- Strategy 1: Facility Median (if 1 or 2 missing years) ---
        # Check if 'facility_id' exists before grouping
        if 'facility_id' in df.columns:
            # Calculate non-null counts per facility for the current column
            # Using transform keeps the original DataFrame shape for easy masking
            facility_counts = df.groupby('facility_id')[col].transform('count')

            # Identify rows where the value is missing AND the facility has 3 or 4 non-null values (1 or 2 missing out of 5 years)
            mask_facility_median = df[col].isnull() & (facility_counts >= 3) & (facility_counts <= 4)

            if mask_facility_median.any():
                # Calculate the median for facilities that meet the criteria
                facility_medians = df.groupby('facility_id')[col].transform('median')
                # Fill the missing values using the mask and the calculated facility medians
                df.loc[mask_facility_median, col] = facility_medians[mask_facility_median]
                # print(f"  Filled {mask_facility_median.sum()} missing values in '{col}' with facility median.") # Optional detailed print
        else:
            print(f"Warning: 'facility_id' column not found. Skipping facility-level imputation for '{col}'.")


        # --- Strategy 2 & 3: County/State Median (if > 2 missing years or remaining NaNs) ---
        # Identify remaining NaNs after attempting facility median imputation
        mask_remaining_nans = df[col].isnull()

        if mask_remaining_nans.any():
            # print(f"  Handling remaining NaNs in '{col}' with county/state median fallback.") # Optional detailed print

            # Calculate county median per year for the current column
            # Ensure 'Year' in df.columns and 'County' in df.columns before grouping
            if 'Year' in df.columns and 'County' in df.columns:
                # Calculate median only for groups with non-null County values
                county_medians = df.groupby(['Year', 'County'])[col].transform(lambda x: x.median() if not x.isnull().all() else np.nan)
                # Fill the remaining NaNs using the mask and county medians
                # This will only fill where county_medians is not NaN itself
                df.loc[mask_remaining_nans, col] = county_medians[mask_remaining_nans]
                # print(f"    Attempted fill with county median. Remaining NaNs: {df[col].isnull().sum()}") # Optional detailed print
            else:
                 print(f"    Warning: 'Year' or 'County' column not found. Skipping county-level imputation for '{col}'.")


            # Identify any NaNs still remaining (where county median might have been NaN)
            mask_still_remaining_nans = df[col].isnull()

            if mask_still_remaining_nans.any():
                 # Calculate state median per year for the current column
                 # Ensure 'Year' in df.columns and 'State Code' in df.columns before grouping
                 if 'Year' in df.columns and 'State Code' in df.columns:
                    # Calculate median only for groups with non-null State Code values
                    state_medians = df.groupby(['Year', 'State Code'])[col].transform(lambda x: x.median() if not x.isnull().all() else np.nan)
                    # Fill the remaining NaNs using the mask and state medians
                    df.loc[mask_still_remaining_nans, col] = state_medians[mask_still_remaining_nans]
                    # print(f"    Attempted fill with state median. Remaining NaNs: {df[col].isnull().sum()}") # Optional detailed print
                 else:
                    print(f"    Warning: 'Year' or 'State Code' column not found. Cannot use state median fallback for '{col}'.")
            # else: # Optional detailed print
                 # print(f"    No NaNs remaining after county median fill for '{col}'.")


    final_missing_count = df[cols_to_impute].isnull().sum().sum()
    print(f"Final total missing values in numeric columns after imputation: {final_missing_count}")
    print("--- Missing Value Imputation Complete ---")

    return df


def save_dataframe(df, output_path):
    """
    Saves the DataFrame to a CSV file at the specified output path.

    Args:
        df (pd.DataFrame): The DataFrame to save.
        output_path (str): The full path (including filename) to save the CSV.
    """
    if not df.empty:
        try:
            df.to_csv(output_path, index=False)
            print(f"\n✅ Final Processed and Integrated DataFrame saved to: {output_path}")
        except Exception as e:
            print(f"\n❌ Error saving DataFrame to {output_path}: {e}")
    else:
        print("\n❌ DataFrame is empty. Cannot save to CSV.")


# --- Combined Pipeline Execution ---

# Step 1: Load and combine raw data from yearly files
df_master = load_and_combine_data(DATA_PATH, YEARS)

# Check if data was loaded successfully before proceeding
if not df_master.empty:

    # Step 2: Clean column names (strip whitespace)
    df_master = clean_column_names(df_master)

    # Step 3: Clean Data Values (remove leading/trailing ',', '.', ' ' from strings)
    df_master = clean_data_values(df_master)

    # Step 4: Drop specified columns (initial drop of less relevant columns)
    df_master = drop_specified_columns(df_master, INITIAL_COLS_TO_DROP)

    # Step 5: Drop rows with missing CBSA number (essential for geographic analysis)
    df_master = drop_rows_missing_cbsa(df_master)

    # Step 6: Create Derived Financial Columns (Total Staffing FTE and Total Revenue)
    df_master = create_derived_financial_columns(df_master)

    # Step 7: Create a unique facility ID for tracking hospitals across years
    df_master = create_facility_id(df_master)

    # Step 8: Filter for facilities reporting in all specified years for consistent time series analysis
    # All subsequent steps will operate on this smaller, filtered DataFrame.
    df_filtered = filter_facilities_all_years(df_master, YEARS)

    # Check if the filtering resulted in an empty DataFrame
    if not df_filtered.empty:

        # Step 9: Rename columns based on the RENAME_MAP
        print("\n--- Step 9: Renaming Columns (Initial and Specific) ---")
        df_renamed = rename_columns(df_filtered.copy(), RENAME_MAP) # Work on a copy to avoid modifying df_filtered


        # --- Data Integration ---
        # Step 10: Load and Process Inflation Data
        df_inflation_processed = load_and_process_inflation_data(INFLATION_DATA_FILE)

        # Step 11: Load and Process Medicare Enrollment Data
        df_medicare_processed = load_and_process_medicare_data(PROCESSED_MEDICARE_FILE)

        # Step 12: Merge Inflation Data with the main DataFrame on 'Year'
        df_integrated = df_renamed.copy() # Start with the renamed hospital data
        if not df_inflation_processed.empty and 'Year' in df_integrated.columns and 'Year' in df_inflation_processed.columns:
            print("--- Step 12: Merging Inflation Data ---")
            # Ensure 'Year' is integer type for merging
            df_integrated['Year'] = df_integrated['Year'].astype(int)
            df_inflation_processed['Year'] = df_inflation_processed['Year'].astype(int)
            df_integrated = pd.merge(df_integrated, df_inflation_processed, on='Year', how='left')
            print("Inflation data merged.")
        elif not df_inflation_processed.empty:
             print("--- Step 12: Skipping Inflation merge: 'Year' column missing in one or both DataFrames ---")
        else:
             print("--- Step 12: Skipping Inflation merge: Processed Inflation Data DataFrame is empty ---")


        # Step 13: Merge Medicare Enrollment Data with the main DataFrame on 'Year' and 'State Code'
        if not df_medicare_processed.empty and 'Year' in df_integrated.columns and 'State Code' in df_integrated.columns and 'Year' in df_medicare_processed.columns and 'State Code' in df_medicare_processed.columns:
            print("--- Step 13: Merging Medicare Enrollment Data ---")
            # Ensure merge columns are of compatible types before merging
            df_integrated['Year'] = df_integrated['Year'].astype(int)
            df_integrated['State Code'] = df_integrated['State Code'].astype(str) # Ensure State Code is string for merging
            df_medicare_processed['Year'] = df_medicare_processed['Year'].astype(int)
            # State Code in medicare_processed is already string from processing

            df_integrated = pd.merge(df_integrated, df_medicare_processed, on=['Year', 'State Code'], how='left')
            print("Medicare Enrollment data merged.")
        elif not df_medicare_processed.empty:
            print("--- Step 13: Skipping Medicare Enrollment merge: Required merge columns ('Year' or 'State Code') missing in one or both DataFrames ---")
        else:
            print("--- Step 13: Skipping Medicare Enrollment merge: Processed Medicare Enrollment Data DataFrame is empty ---")

        print("--- Data Integration Complete ---")


        # Step 14: Select the final financial subset columns (including integrated data)
        # Create the final list of columns to select, including the new ones from external data
        final_columns_to_select = COLUMNS_FOR_FINAL_SUBSET.copy()
        # Add columns from external data if they exist after merging
        if 'Annual' in df_integrated.columns:
             final_columns_to_select.append('Annual')
        if 'Total_Yearly_Medicare_Enrollment' in df_integrated.columns:
             final_columns_to_select.append('Total_Yearly_Medicare_Enrollment')

        print("\n--- Step 14: Selecting Final Financial Subset Columns ---")
        # Ensure all columns in final_columns_to_select are actually in df_integrated before selecting
        # This handles cases where a column might not have been created (e.g., if external data file was missing)
        present_final_columns = [col for col in final_columns_to_select if col in df_integrated.columns]
        df_final_subset = df_integrated[present_final_columns].copy() # Use .copy()
        print(f"Selected {len(present_final_columns)} columns for the final subset.")
        # Report any columns requested but not found
        missing_final_columns = [col for col in final_columns_to_select if col not in df_integrated.columns]
        if missing_final_columns:
             print(f"Warning: The following columns were requested for the final subset but not found in the DataFrame: {missing_final_columns}")


        # Step 15: Impute missing values in the final subset using defined strategies
        df_final_imputed = impute_missing_financial_data(df_final_subset)


        # Step 16: Update Categorization Dictionary with new columns for documentation/analysis purposes
        # Add entries for the new columns from external data to the global map
        # Also ensure the renamed column is in the map if it wasn't already
        if 'Annual' in df_final_imputed.columns and 'Annual' not in column_category_map:
             column_category_map['Annual'] = {'category': 'External Factors', 'subcategory': 'Inflation'}
             print("Added 'Annual' to categorization map.")
        if 'Total_Yearly_Medicare_Enrollment' in df_final_imputed.columns and 'Total_Yearly_Medicare_Enrollment' not in column_category_map:
             column_category_map['Total_Yearly_Medicare_Enrollment'] = {'category': 'External Factors', 'subcategory': 'Medicare Enrollment'}
             print("Added 'Total_Yearly_Medicare_Enrollment' to categorization map.")
        # Ensure the target renamed column is in the map if it wasn't already
        if 'Hospital Total Days Or Visits' in df_final_imputed.columns and 'Hospital Total Days Or Visits' not in column_category_map:
             # Use the category/subcategory from the original column if available, or set a default
             original_col_name = "Hospital Total Days (V + XVIII + XIX + Unknown) For Adults & Peds"
             if original_col_name in column_category_map:
                 column_category_map['Hospital Total Days Or Visits'] = column_category_map[original_col_name]
             else:
                 column_category_map['Hospital Total Days Or Visits'] = {'category': 'Operations', 'subcategory': 'Capacity'} # Default
             print("Ensured 'Hospital Total Days Or Visits' is in categorization map.")


        # --- Using the Categorization (with updated map) ---
        # This section demonstrates how to use the categorization dictionary.
        print("\n--- Using the Categorization Dictionary with Final Data ---")

        # Example: Get all columns in the 'External Factors' category within the final subset
        external_cols_in_subset = [col for col in df_final_imputed.columns if col in column_category_map and column_category_map[col]['category'] == 'External Factors']
        print(f"Columns in the 'External Factors' category within the final subset: {external_cols_in_subset}")

        # Example: Iterate through main categories and list columns present in the final subset
        print("\nColumns in Final Subset by Category (using updated map):")
        # Get unique categories from the updated map
        categories = set(info['category'] for info in column_category_map.values())
        for category in sorted(list(categories)):
            # Find columns in the final DataFrame that belong to this category
            cols_in_category = [col for col in df_final_imputed.columns if col in column_category_map and column_category_map[col]['category'] == category]
            # Further filter to only include columns actually present in the final DataFrame
            present_cols_in_category = [col for col in cols_in_category if col in df_final_imputed.columns]
            if present_cols_in_category:
                print(f"- {category}: {present_cols_in_category}")


        # Step 17: Save the final processed, integrated, and imputed DataFrame to a CSV file
        save_dataframe(df_final_imputed, FINAL_OUTPUT_FILE_PATH)

    else:
        print("\nFiltering resulted in an empty DataFrame. No further processing will be done.")

else:
    print("Pipeline stopped because no data was loaded.")



--- Step 1: Loading and Combining Data ---
Successfully loaded data for year 2018.
Successfully loaded data for year 2019.
Successfully loaded data for year 2020.
Successfully loaded data for year 2021.
Successfully loaded data for year 2022.
Combined data from 5 years into a master DataFrame (initial load).
--- Step 2: Cleaning Column Names ---
Stripped whitespace from column names.
--- Step 3: Cleaning Data Values ---
Removed leading/trailing characters (','. ') from string columns.
--- Step 4: Dropping Specified Columns ---
Dropped columns: ['Hospital Total Days Title V For Adults & Peds', 'Hospital Total Days Title XVIII For Adults & Peds', 'Hospital Total Days Title XIX For Adults & Peds']
--- Step 5: Dropping Rows with Missing CBSA ---
Dropped 382 rows with missing or empty 'Medicare CBSA Number'. Remaining rows: 30075
--- Step 6: Creating Derived Financial Columns (Staffing FTE, Total Revenue) ---
  Combining staffing FTE columns: ['FTE - Employees on Payroll', 'Number of Intern